In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import gspread
from oauth2client.service_account import ServiceAccountCredentials

app = Flask(__name__)
CORS(app)

# Google Sheets authentication
scope = ["https://spreadsheets.google.com/feeds",
         "https://www.googleapis.com/auth/drive"]

creds = ServiceAccountCredentials.from_json_keyfile_name("credentials.json", scope)
client = gspread.authorize(creds)

# Open your sheet
SPREADSHEET_ID = "1zhMAONxk6Hz0Hz5EZFxCf-NdfwYlYRezW5War-mjHH8"  
sheet = client.open_by_key(SPREADSHEET_ID).worksheet("DB_Sheet")

# Fetch all products
@app.route("/products", methods=["GET"])
def get_products():
    try:
        data = sheet.get_all_records()
        return jsonify(data)
    except Exception as e:
        return jsonify({"error": str(e)}), 500

# Update rating
@app.route("/update-rating", methods=["POST"])
def update_rating():
    try:
        req = request.get_json()
        product_id = str(req.get("id"))
        new_rating = req.get("newRating")

        if not product_id or new_rating is None:
            return jsonify({"error": "Missing id or newRating"}), 400

        records = sheet.get_all_records()
        for idx, row in enumerate(records, start=2):  # start=2 (row 1 is headers)
            if str(row["id"]) == product_id:
                sheet.update_cell(idx, 9, new_rating)  # column 9 = rating
                return jsonify({"success": True, "id": product_id, "newRating": new_rating})

        return jsonify({"error": "Product not found"}), 404
    except Exception as e:
        return jsonify({"error": str(e)}), 500

# Add new product
@app.route("/add-product", methods=["POST"])
def add_product():
    try:
        req = request.get_json()

        new_row = [
            req.get("id"),
            req.get("date"),
            req.get("category"),
            req.get("unit_measurement"),
            req.get("price"),
            req.get("freshness"),
            req.get("vendor"),
            req.get("location"),
            req.get("rating"),
            req.get("image_url"),
        ]

        # Ensure no missing required fields
        if any(v is None for v in new_row):
            return jsonify({"error": "Missing required fields"}), 400

        sheet.append_row(new_row)
        return jsonify({"success": True, "product": new_row})
    except Exception as e:
        return jsonify({"error": str(e)}), 500

if __name__ == "__main__":
    #app.run(port=4000, debug=True)
    app.run(port=4000, debug=False, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:4000
Press CTRL+C to quit
127.0.0.1 - - [26/Aug/2025 01:53:04] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [26/Aug/2025 01:53:04] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [26/Aug/2025 01:53:35] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [26/Aug/2025 01:53:37] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [26/Aug/2025 01:54:35] "GET /products HTTP/1.1" 200 -


In [ ]:
import requests
# Read Everything
resp = requests.get("http://127.0.0.1:4000/products")
print(resp.json())


#Update rating (for product with id=1 for example):
resp = requests.post("http://127.0.0.1:4000/update-rating", json={"id": 1, "newRating": 3})
print(resp.json())

#Add a new product:
resp = requests.post("http://127.0.0.1:4000/add-product", json={
    "id": 20,
    "date": "2025-08-26",
    "category": "Tomatoes",
    "unit_measurement": "KG",
    "price": "50/KG",
    "freshness": "Today",
    "vendor": "Vendor Z",
    "location": "CBD",
    "rating": 5,
    "image_url": "src/assets/Tomatoes.jfif"
})
print(resp.json())


#If you want, we can add a welcome route like:
@app.route("/")
def index():
    return "Backend is running! 🚀"

Working dir: c:\Users\user\Python_projects\React_Projects\githu_project\Backend
File exists: True
